In [ ]:
#  Copyright (c) 2017-2025 KAPPA
#
#  Licensed under the Apache License, Version 2.0 (the "License");
#  you may not use this file except in compliance with the License.
#  You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
#  Unless required by applicable law or agreed to in writing, software
#  distributed under the License is distributed on an "AS IS" BASIS,
#  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#  See the License for the specific language governing permissions and
#  limitations under the License.
#
# This file is part of Orchid and related technologies.
#

# Example: Plotting well time series

This notebook illustrates using the Orchid* Python API and the pandas package to
plot well time series.

(*Orchid is a mark of KAPPA)

## 0.5 Import packages

The only import needed for the Python API is `orchid` itself.

In [ ]:
import orchid

The remaining imports are standard python packages to support the analysis.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
# The following import is included for its "side-effects" of an improved color schemes and
# plot styles. (See the "Tip" in section 9.2 of "Python for Data Analysis" for details.)
import seaborn as sns

## 1.0 Load the .ifrac project

The following code simply captures the configured location of the Orchid training data. It is not needed to
use the Orchid Python API itself, but it is used in this example to load well-known data.

In [ ]:
orchid_training_data_path = orchid.training_data_path()

In [ ]:
project = orchid.load_project(str(orchid_training_data_path.joinpath(
    'Project_frankNstein_Permian_UTM13_FEET.ifrac')))

### 1.1 Get the time series (be patient)

In [ ]:
all_time_series = list(project.time_series().all_objects())
all_time_series_display_names = list(project.time_series().all_display_names())
data_points = [series.data_points() for series in all_time_series]

### 1.2 Plot the time series in a 4x4 array

Define a helper functions to plot a single series

In [ ]:
def plot_time_series(using_series, on_axes, using_color, physical_quantity,
                     unit_abbreviation, name):
    """
    Plot the specified time series using the supplied details

    Args:
        using_series: The series to plot.
        on_axes: The axes on which to plot the series.
        using_color: The color for the plot.
        physical_quantity: The physical quantity of the series.
        unit_abbreviation: The abbreviation of the unit_abbreviation of the plotted samples.
        name: The name of the curve.
    """
    using_series.plot(ax=on_axes, color=using_color)
    on_axes.set_ylabel(f'{str(physical_quantity).title()} ({unit_abbreviation})')
    on_axes.title.set_text(name)
    x_tick_labels = on_axes.get_xticklabels()
    plt.setp(x_tick_labels, rotation=30)

In [ ]:
default_well_colors = ['#%02x%02x%02x' % tuple(map(lambda c: int(255 * c), (r, g, b)))
                       for (r, g, b) in project.default_well_colors()]
plot_series_shape = (2, 2)
max_curve_count = plot_series_shape[0] * plot_series_shape[1]
figure, axes = plt.subplots(*plot_series_shape, figsize=(10, 14), dpi=200)
time_series_to_plot = np.reshape(all_time_series[:max_curve_count], plot_series_shape)
data_points_to_plot = np.reshape(np.array(data_points[:max_curve_count], dtype=object), plot_series_shape)
names_to_display = np.reshape(all_time_series_display_names[:max_curve_count], plot_series_shape)
colors_to_use = np.reshape(default_well_colors[:max_curve_count], plot_series_shape)
for i in range(len(axes)):
    for j in range(len(axes[0])):
        series = time_series_to_plot[i, j]
        points = data_points_to_plot[i, j]
        ax = axes[i, j]
        color = colors_to_use[i, j]
        quantity_name = series.sampled_quantity_name
        abbreviation = orchid.abbreviation(series.sampled_quantity_unit())
        series_name = names_to_display[i, j]
        plot_time_series(points, ax, color, quantity_name, abbreviation, series_name)

plt.show()